In [87]:
import numpy
import random
from PIL import Image
import pandas as pd
import numpy as np
import cv2
import math
import os

In [181]:
files = []
max_size = 65536
df = pd.read_csv('sizes.csv','r',delimiter=',')


for index,row in df.iterrows():
    if row['size']>5000 and row['size']<max_size:
        files.append(row['name'])
print(len(files))        

2303


In [184]:
def create_image(name):
    df = pd.read_csv("decimal_files/" + name,sep='|',error_bad_lines=False,warn_bad_lines=False,engine='python')

    decimals = []
    for index, row in df.iterrows():
        values = row['bytes'].split('-')
        if row['type']=='data':
            values = [value+'d' for value in values]
        else:
            values = [value+'c' for value in values]
        decimals.extend(values)
    
    length = len(decimals)
    dimension = math.ceil(math.sqrt(length))
    bw_data = numpy.zeros((dimension, dimension), dtype=numpy.uint8)
    col_data = numpy.zeros((dimension, dimension, 3), dtype=numpy.uint8)
    
    for num in range(length):
        if decimals[num][-1]=='c':
            col = [0,255,0]
            gray = 85
        else:
            col = [255,0,0]
            gray = 170
        if (num//dimension%2==0):

            bw_data[num//dimension,num%dimension] = decimals[num][:-1]
            col_data[num//dimension,num%dimension] = col
        else:
            bw_data[num//dimension,dimension - num%dimension - 1]=decimals[num][:-1]
            col_data[num//dimension,dimension - num%dimension - 1]=col
            
    image = Image.fromarray(bw_data)
    image.save("images/bw256/"+name.strip('csv')+"png",quality=100)
    image = Image.fromarray(col_data)
    image.save("images/col256/"+name.strip('csv')+"png",quality=100)
 

In [ ]:
Starting and .bss are 255
Extern is 0. We can pad with 255 at the start

In [185]:
for count,file in enumerate(files):
    create_image(file)
    if count%100==0:
        print(count)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300


In [188]:
#resize images

path1 = '/Users/aizazansari/Desktop/Uni Data/MOMA/images/bw256/'
path2 = '/Users/aizazansari/Desktop/Uni Data/MOMA/images/col256/'
dim = int(math.sqrt(max_size))
arr = os.listdir(path1)

for img in arr:
    if img.endswith('.png'):
        image1 = Image.open(path1+img)
        ret_image1 = image1.resize([dim,dim], Image.NEAREST)
        ret_image1.save("images_fixed/bw256/"+ img,quality=100)

        image2 = Image.open(path2+img)
        ret_image2 = image2.resize([dim,dim], Image.NEAREST)
        ret_image2.save("images_fixed/col256/"+ img,quality=100)

256.0

True

In [193]:
color2index = {(0, 0, 0): 0,
 (0, 255, 0): 1,
 (255, 0, 0): 2}

In [194]:
def rgb_to_index(name):
    index_data = numpy.zeros((128, 128), dtype=numpy.uint8)
    rgb_data = np.array(Image.open('images_fixed/col/'+name))
    for i in range(128):
        for j in range(128):
            index_data[i][j] = color2index[tuple(rgb_data[i][j])]
    img = Image.fromarray(index_data)
    img.save('images_fixed/index/'+name)

'decimal226.png'

In [195]:
#generate index images
path = '/Users/aizazansari/Desktop/Uni Data/MOMA/images_fixed/col'
arr = os.listdir(path)
for name in arr:
    if name.endswith('.png'):
        rgb_to_index(name)